In [6]:
%%capture
%run preprocessing.ipynb

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# Assuming that your data is ready and is in the variables X_train, X_test, y_train, y_test
# Load data
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy') - 1 # subtract 1 to make the labels 0, 1, 2 
y_test = np.load('y_test.npy') -1    # same thing here

# Convert your data into PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create Tensor datasets
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)  # amount of data to be loaded each time
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

# Define the model
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 32),
    nn.ReLU(),
    nn.Linear(32, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 3),
    nn.Softmax(dim=1)
)

# Define the loss and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

# Train the model
for epoch in range(100):
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

# Evaluate the model
model.eval()
with torch.no_grad():
    all_labels = []
    all_predictions = []
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)   # get the index of the max log-probability
        all_labels.extend(labels.numpy())
        all_predictions.extend(predicted.numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')

# Print metrics
print('Accuracy: {}'.format(accuracy))
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1 Score: {}'.format(f1))


Accuracy: 0.9773818133999606
Precision: 0.9773401825089539
Recall: 0.9773818133999606
F1 Score: 0.9773343861990973


In [ ]:
import random
from sklearn.model_selection import ParameterGrid
from torch.optim import Adam, SGD, RMSprop, Adagrad, Adamax, Adadelta
import numpy as np
import pandas as pd
import time

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Load data
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy') - 1 # subtract 1 to make the labels 0, 1, 2 
y_test = np.load('y_test.npy') -1    # same thing here

# Convert your data into PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create Tensor datasets
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

# Define the hyperparameters
hyperparameters = {
    'learning_rate': [0.1, 0.01, 0.001, 0.0001],
    'batch_size': [16, 32, 64],
    'num_epochs': [50, 100, 200, 300],
    'optimizer': [Adam, SGD, RMSprop, Adagrad, Adamax, Adadelta]
}

# Create a list to store the results
results = []

# Create a grid of hyperparameters
grid = list(ParameterGrid(hyperparameters))

# Convert the grid to a list and shuffle it to randomize the order of combinations
random.shuffle(grid)

# Use the first 60 combinations (make sure there are no duplicates)
sampler = grid[:60]

best_f1 = 0
best_hyperparameters = None

# Iterate over each combination of hyperparameters
for i, params in enumerate(sampler, start=1):
    # Start the timer
    start_time = time.time()

    # Create a new model with the current hyperparameters
    model = nn.Sequential(
        nn.Linear(X_train.shape[1], 32),
        nn.ReLU(),
        nn.Linear(32, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 16),
        nn.ReLU(),
        nn.Linear(16, 3),
        nn.Softmax(dim=1)
    )

    # Define the loss and the optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = params['optimizer'](model.parameters(), lr=params['learning_rate'])

    # Create DataLoaders with the current batch size
    train_loader = DataLoader(train_data, batch_size=params['batch_size'], shuffle=True)
    test_loader = DataLoader(test_data, batch_size=params['batch_size'], shuffle=True)

    # Train the model for the specified number of epochs
    for epoch in range(params['num_epochs']):
        for inputs, labels in train_loader:
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()
    
    # Evaluate the model
    model.eval()
    with torch.no_grad():
        all_labels = []
        all_predictions = []
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)   # get the index of the max log-probability
            all_labels.extend(labels.numpy())
            all_predictions.extend(predicted.numpy())

    # Stop the timer
    end_time = time.time()

    # Calculate training time
    training_time = end_time - start_time

    # Print the number of models trained so far
    print(f"Trained {i} out of 60 models.")

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')

    # Save the results
    results.append({
        'learning_rate': params['learning_rate'],
        'batch_size': params['batch_size'],
        'num_epochs': params['num_epochs'],
        'optimizer': params['optimizer'].__name__,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'training_time': training_time
    })

    # If the current model is better than all previous models, store its F1 score and hyperparameters
    if f1 > best_f1:
        best_f1 = f1
        best_hyperparameters = params

# Print the best accuracy and hyperparameters
print('Best F1: {}'.format(best_f1))
print('Best Hyperparameters: {}'.format(best_hyperparameters))

# Convert the results to a dataframe
results_df = pd.DataFrame(results)
# Save the dataframe to a csv file
results_df.to_csv('Results_Neural_Networks.csv', index=False)